## Work on bus delays

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_list = list()

for i in range(11):
    data = pd.read_excel('/Users/laurabresson/Data_projects/Is_TTC_really_bad?/raw_data/bus-data2019.xlsx', sheet_name= i)
    df_list.append(data)

bus2019 = pd.concat(df_list)
bus2019.sample(10)

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Incident ID,Delay,Gap
2338,2019-10-18,924,10:16:00,Friday,Eglinton/Rosemount,Investigation,1.0,1.0,E/B,9215.0,NaN,NaN,NaN
1375,2019-06-07,88,16:30:00,Friday,Entire 88 route bothways,General Delay,NaN,NaN,B/W,8306.0,NaN,12.0,18.0
2841,2019-06-16,161,09:28:00,Sunday,Jane and Alliance,Mechanical,NaN,NaN,S/B,1272.0,NaN,1.0,1.0
4466,2019-01-27,38,04:00:00,Sunday,Highland Creek Route,General Delay,12.0,24.0,B/W,NaN,NaN,NaN,NaN
1086,2019-02-05,57,08:26:00,Tuesday,Midland Ave/Midwest Rd,Mechanical,8.0,17.0,N/B,8857.0,NaN,NaN,NaN
3789,2019-04-23,939,07:00:00,Tuesday,McCowan under SRT Bridge,Diversion,NaN,NaN,S/B,1668.0,3.0,420.0,429.0
3003,2019-01-21,118,05:40:00,Monday,Arrow garage POD Wilson stn,Late Leaving Garage,20.0,40.0,W/B,3229.0,NaN,NaN,NaN
802,2019-02-03,38,23:26:00,Sunday,STC,Investigation,13.0,26.0,E/B,1561.0,NaN,NaN,NaN
3856,2019-01-24,927,06:33:00,Thursday,KPSN,Mechanical,3.0,9.0,N/B,3368.0,NaN,NaN,NaN
2115,2019-07-14,102,10:41:00,Sunday,Kennedy and St.Clair,Diversion,37.0,47.0,N/B,8522.0,NaN,NaN,NaN


In [3]:
bus2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57892 entries, 0 to 5435
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Report Date  57892 non-null  datetime64[ns]
 1   Route        57892 non-null  int64         
 2   Time         57892 non-null  object        
 3   Day          57892 non-null  object        
 4   Location     57721 non-null  object        
 5   Incident     56957 non-null  object        
 6   Min Delay    41924 non-null  float64       
 7   Min Gap      41836 non-null  float64       
 8   Direction    56772 non-null  object        
 9   Vehicle      46519 non-null  float64       
 10  Incident ID  5144 non-null   float64       
 11  Delay        15732 non-null  float64       
 12  Gap          15717 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(5)
memory usage: 6.2+ MB


----
----
### Data wrangling and dealing with NaN 

In [4]:
bus2019.isna().sum() / len(bus2019) *100

Report Date     0.000000
Route           0.000000
Time            0.000000
Day             0.000000
Location        0.295378
Incident        1.615076
Min Delay      27.582395
Min Gap        27.734402
Direction       1.934637
Vehicle        19.645201
Incident ID    91.114489
Delay          72.825261
Gap            72.851171
dtype: float64

We first need to combine the two pairs of columns representing the values for delays and gaps.

In [5]:
# combining Min Delay & Delay by creating a new column 
bus2019['Delays'] = np.where(bus2019['Delay'].isna(), bus2019['Min Delay'], bus2019['Delay'])
                    # si 'Delay' is a NaN, look up in 'Min Delay', otherwise keep original value

# combining Min Gap & Gap by creating a new column
bus2019["Gaps"] = np.where(bus2019["Gap"].isna(), bus2019["Min Gap"], bus2019["Gap"])
                    # si 'Gap' is a NaN, look up in 'Min Gap', otherwise keep original value
    
# dropping the original columns now 
bus2019.drop(columns= ['Min Delay', 'Min Gap', 'Gap', 'Delay'],\
            inplace= True)

# checking result
print(bus2019.isna().sum() / len(bus2019) *100)
print("----")
bus2019.sample(5)

Report Date     0.000000
Route           0.000000
Time            0.000000
Day             0.000000
Location        0.295378
Incident        1.615076
Direction       1.934637
Vehicle        19.645201
Incident ID    91.114489
Delays          0.407656
Gaps            0.585573
dtype: float64
----


,Report Date,Route,Time,Day,Location,Incident,Direction,Vehicle,Incident ID,Delays,Gaps
5094,2019-04-30,53,16:05:00,Tuesday,Passmore and Markham,Investigation,W/B,1688.0,5.0,6.0,11.0
1088,2019-02-05,925,08:30:00,Tuesday,PSTN to FMDM,General Delay,B/W,NaN,NaN,20.0,30.0
1391,2019-09-12,161,06:00:00,Thursday,Humber Blvd at Cliff,Diversion,E/B,NaN,NaN,999.0,999.0
2547,2019-01-18,65,18:36:00,Friday,CAST,Mechanical,S/B,9237.0,NaN,14.0,29.0
2475,2019-06-13,131,20:21:00,Thursday,Nugget and Markham,Mechanical,EB,1401.0,NaN,13.0,26.0


In [6]:
# dropping 2 other columns quite useless 
# Vehicle and Incident ID 
bus2019.drop(columns= ['Incident ID', 'Vehicle'],\
            inplace= True)
bus2019.sample(5)

,Report Date,Route,Time,Day,Location,Incident,Direction,Delays,Gaps
4952,2019-07-31,116,00:30:00,Wednesday,Kennedy Stn,Utilized Off Route,E/B,10.0,20.0
2998,2019-04-18,26,07:50:00,Thursday,Dupont Route,General Delay,B/W,15.0,29.0
2726,2019-01-19,122,18:40:00,Saturday,Graydon Hall & Fenelon,Investigation,E/B,30.0,60.0
1543,2019-06-08,101,13:05:00,Saturday,Chesswood,Diversion,W/B,55.0,75.0
1992,2019-02-08,168,08:00:00,Friday,DWSTN,General Delay,B/W,362.0,372.0


In [7]:
bus2019.isna().sum() / len(bus2019) *100

Report Date    0.000000
Route          0.000000
Time           0.000000
Day            0.000000
Location       0.295378
Incident       1.615076
Direction      1.934637
Delays         0.407656
Gaps           0.585573
dtype: float64

To deal with the rest of the NaN, there's the need to fill the remaining NaN. 

NaN to be filled with 'Unknown' for columns Location, Incident and Direction. 

In [8]:
# 1. filling NaN with 0 
bus2019.Location.fillna(value= 0, inplace= True)
bus2019.Incident.fillna(value= 0, inplace= True)
bus2019.Direction.fillna(value= 0, inplace= True)

# 2. replacing 0 with 'Unknown'
bus2019.Location.replace(to_replace= 0, value= 'Unknown', inplace= True)
bus2019.Incident.replace(to_replace= 0, value= 'Unknown', inplace= True)
bus2019.Direction.replace(to_replace= 0, value= 'Unknown', inplace= True)

# checking
bus2019.isna().sum()

Report Date      0
Route            0
Time             0
Day              0
Location         0
Incident         0
Direction        0
Delays         236
Gaps           339
dtype: int64

NaN to be filled with either the mean or median for Delays and Gaps.

In [9]:
# 1. calculating the mean of the Delays and Gaps
delays_mean = bus2019.Delays.mean()
gaps_mean = bus2019.Gaps.mean()

# 2. filling NaN with the median 
bus2019.Delays.fillna(value= delays_mean, inplace= True)
bus2019.Gaps.fillna(value= gaps_mean, inplace= True)

# checking
bus2019.isna().sum() 

Report Date    0
Route          0
Time           0
Day            0
Location       0
Incident       0
Direction      0
Delays         0
Gaps           0
dtype: int64

---
---
### Data wrangling - feature engineering 

What is possible and interesting to know more about ? 

In [10]:
print(bus2019.dtypes)
print("----")
bus2019.sample(5)

Report Date    datetime64[ns]
Route                   int64
Time                   object
Day                    object
Location               object
Incident               object
Direction              object
Delays                float64
Gaps                  float64
dtype: object
----


,Report Date,Route,Time,Day,Location,Incident,Direction,Delays,Gaps
817,2019-01-07,36,18:12:00,Monday,Toroyork and Ormont,Investigation,W/B,13.0,26.0
5728,2019-01-29,41,20:00:00,Tuesday,Weston at St clair,Emergency Services,B/W,10.0,20.0
2951,2019-06-17,143,07:55:00,Monday,Richmond St East and Bay St,Investigation,WB,15.0,30.0
1878,2019-04-12,134,07:00:00,Friday,STC--FDFI-IRON-CECO-STC,General Delay,B/W,18.0,36.0
1229,2019-09-11,37,07:00:00,Wednesday,ENTIRE ROUTE,General Delay,B/W,14.0,35.0


Extracting more information out of the Report Date
- quarter
- month
- day of the month
- hour

In [11]:
# extracting facets and putting them in separate columns
bus2019['Report Quarter'] = pd.DatetimeIndex(bus2019['Report Date']).quarter
bus2019['Report Month'] = pd.DatetimeIndex(bus2019['Report Date']).month
bus2019['Report Day'] = pd.DatetimeIndex(bus2019['Report Date']).day

# Time needs a sting approach
bus2019['Time'] = bus2019['Time'].astype('str')
report_hour = bus2019.Time.str.split(':')
bus2019['Report Hour'] = report_hour.str.get(0)

# checking
bus2019.sample(5)

,Report Date,Route,Time,Day,Location,Incident,Direction,Delays,Gaps,Report Quarter,Report Month,Report Day,Report Hour
3994,2019-03-21,10,15:49:00,Thursday,Unknown,Mechanical,S/B,1.0,1.0,1,3,21,15
2846,2019-09-23,6,13:41:00,Monday,Davenport and Dupont,Mechanical,S/B,12.0,24.0,3,9,23,13
1538,2019-08-12,129,16:25:00,Monday,Scarborough Town Center,Utilized Off Route,W/B,5.0,10.0,3,8,12,16
4125,2019-10-31,116,06:16:00,Thursday,EGLI YARD,Mechanical,E/B,1.0,1.0,4,10,31,06
3588,2019-07-22,47,14:34:00,Monday,Lansdowne and Bloor,Investigation,S/B,5.0,10.0,3,7,22,14


Is Direction a mess ?

In [12]:
print(bus2019.Direction.nunique())
print("----")
bus2019.Direction.unique()

131
----


array(['W/B', 'S/B', 'N/B', 'E/B', 'NB', 'bw', 'B/W', 'EB', 'Unknown',
       'BW', 's', 'WB', '66N', 'SB', '5', 'sb', 'SB.', 'n', '8453', 'EW',
       'wb', '37', 'eb', 'sq', 'Bw', 'Wq', 's`', 'BV', 'ewb', 'B', '35',
       'b/w/B', ';22', '18', 'e/b', 'EN', 'RE', 'O/B', '78612', '12',
       's/b', 'OB', 'see', '68134', 'SWB', '07', '52038', 'sn', 'WE', '`',
       'nb', 'Fis', 'N', 'Note', 'Used Rad bus 8180', ';58', 'N/sb', 'E',
       '3', 'v', '69677', 'ns', 'SN', 'n1', 'e/B', 'b/b', '1', 'en',
       'Note:', '7', '23', 'Sb', 'N/W', '15', '8620', 'S8', 'w', 'Bus',
       'bn', 'BN', '320', 'n/b', 'W', 'lEB',
       '@ 16:43 resumed service EB from Arrow A7 bartor', 'eq', '102',
       '20', '19', '58936', '74406', 'sw', 'E/Bb', 'z', 'Operator', 'Uti',
       '53122', 'RAD utilized to partially cover the trip tiomes.',
       'Mino', 'q', 'S', 'N9', 'we', 'All runs cl', 'ew', 'b', '47',
       'All', 'NE', 'Wb', 'WW', 'RAD 8304', 'B?w', 'First', 'Eb',
       'Bus placed back in s

Oh yeah, that's a mess. 

Before making all of these changes which will be inplace= True, making a copy of the data.

In [13]:
buses = bus2019.copy()

In [14]:
# first replacing digits with Unknown
buses.Direction.replace(to_replace= '^\w+?\d+\w?$', value= 'Unknown',\
                         regex= True, inplace= True)

# doing values easily understood as Northbound
buses.Direction.replace(to_replace= '(^[Nn]\/?[Bb]$)|(^[Bb]\/?[Nn]$)', value= 'Northbound',\
                       regex= True, inplace= True)

# doing values easily understood as Eastbound
buses.Direction.replace(to_replace= '(^[Ee]\/?[Bb]$)|(^[Bb]\/?[Ee]$)', value= 'Eastbound',\
                       regex= True, inplace= True)

# doing values easily understood as Southbound
buses.Direction.replace(to_replace= '(^[Ss]\/?[Bb]$)|(^[Bb]\/?[Ss]$)', value= 'Southbound',\
                       regex= True, inplace= True)

# doing values easily understood as Westbound
buses.Direction.replace(to_replace= '(^[Ww]\/?[Bb]$)|(^[Bb]\/?[Ww]$)', value= 'Westbound',\
                       regex= True, inplace= True)

# now that we have the labels we want, everything else can be Unknown
buses.Direction.replace(to_replace= '^((?!.+bound|Unknown).)*$', value= 'Unknown',\
                       regex= True, inplace= True)

print(buses.Direction.nunique())
buses.Direction.unique()

5


array(['Westbound', 'Southbound', 'Northbound', 'Eastbound', 'Unknown'],
      dtype=object)

In [15]:
buses.Direction.value_counts(normalize= True)

Westbound     0.384699
Northbound    0.212586
Southbound    0.194776
Eastbound     0.181061
Unknown       0.026878
Name: Direction, dtype: float64

---
---
### Data wrangling - data types

Let's see if data types correspond to what I want.

In [16]:
bus_delay = buses.copy()

In [17]:
bus_delay.dtypes

Report Date       datetime64[ns]
Route                      int64
Time                      object
Day                       object
Location                  object
Incident                  object
Direction                 object
Delays                   float64
Gaps                     float64
Report Quarter             int64
Report Month               int64
Report Day                 int64
Report Hour               object
dtype: object

| Status | Column | 
| --- | --- |
| ✅ | Report Date/Hour, Time, Day, Location, Direction | 
| --- | --- |
| ❌ | Route, Report Quarter/Month/Day |
| --- | --- |

All ❌ should be object instead of integers. 

In [18]:
# let's try a df.convert_types() 
bus_delay.convert_dtypes()
bus_delay.dtypes

Report Date       datetime64[ns]
Route                      int64
Time                      object
Day                       object
Location                  object
Incident                  object
Direction                 object
Delays                   float64
Gaps                     float64
Report Quarter             int64
Report Month               int64
Report Day                 int64
Report Hour               object
dtype: object

That didn't change anything so let's do it manually.

In [19]:
bus_delay['Route'] = bus_delay['Route'].astype('category')
bus_delay['Report Quarter'] = bus_delay['Report Quarter'].astype('category')
bus_delay['Report Month'] = bus_delay['Report Month'].astype('category')
bus_delay['Report Day'] = bus_delay['Report Day'].astype('category')
bus_delay['Report Hour'] = bus_delay['Report Hour'].astype('category')
bus_delay['Day'] = bus_delay['Day'].astype('category')
bus_delay['Incident'] = bus_delay['Incident'].astype('category')
bus_delay['Direction'] = bus_delay['Direction'].astype('category')
bus_delay['Location'] = bus_delay.Location.astype('string')

# checking 
bus_delay.dtypes

Report Date       datetime64[ns]
Route                   category
Time                      object
Day                     category
Location                  string
Incident                category
Direction               category
Delays                   float64
Gaps                     float64
Report Quarter          category
Report Month            category
Report Day              category
Report Hour             category
dtype: object

I'd like now to export the clean data into a .csv easy to download again. 

In [20]:
# first making a copy
buses_clean = bus_delay.copy()

# now exporting to .csv
buses_clean.to_csv(path_or_buf= '/Users/laurabresson/Data_projects/Is_TTC_really_bad?/clean_data/buses_clean.csv',\
                  index= False)